In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import sparse as sps

/home/vlad/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [28]:
%%bash

cargo run --example sample -- --out test --degree 5 --nvertices 100 --seed 1234

for i in $(seq 1 10) ; do
cargo run --example color -- --graph test.0 --nsamples 10000000 \
  --frequency 1000000 --out colors.txt.$i --out-times times.txt.$i --seed $i
done
  
ls *.txt test.*

{"nedges":599,"nvertices":100,"sample_duration":"2ms"}
{"indexing_duration":"387µs"}
{"nfiles":1,"write_duration":"867µs"}
{"edge_time":"280µs","offset_time":"258µs","slice_build_time":"12µs","sort_time":"209µs"}
{"load_graph_duration":"2ms"}
{"max_degree":21,"nedges":599,"nvertices":100}
{"greedy_color_time":"217µs","greedy_ncolors":7,"vertex_sort_time":"127µs"}
{"conflict_percent":50.9960638078293,"conflicts":10406524,"glauber_ncolors":43,"greedy_ncolors":7,"nsamples":10000000,"nthreads":8}
{"color_cardinalities":{"mean":2.5,"p0.000":1.0,"p0.001":1.0,"p0.010":1.0,"p0.050":1.0,"p0.100":1.0,"p0.500":2.0,"p0.900":4.0,"p0.950":5.0,"p0.990":5.0,"p1.000":6.0},"colors_duration":"19s","ncolors":43}
{"edge_time":"338µs","offset_time":"311µs","slice_build_time":"15µs","sort_time":"247µs"}
{"load_graph_duration":"2ms"}
{"max_degree":21,"nedges":599,"nvertices":100}
{"greedy_color_time":"256µs","greedy_ncolors":7,"vertex_sort_time":"143µs"}
{"conflict_percent":51.05148507591596,"conflicts":10429

    Finished dev [unoptimized + debuginfo] target(s) in 0.02s
     Running `target/debug/examples/sample --out test --degree 5 --nvertices 100 --seed 1234`
    Finished dev [unoptimized + debuginfo] target(s) in 0.02s
     Running `target/debug/examples/color --graph test.0 --nsamples 10000000 --frequency 1000000 --out colors.txt.1 --out-times times.txt.1 --seed 1`
    Finished dev [unoptimized + debuginfo] target(s) in 0.02s
     Running `target/debug/examples/color --graph test.0 --nsamples 10000000 --frequency 1000000 --out colors.txt.2 --out-times times.txt.2 --seed 2`
    Finished dev [unoptimized + debuginfo] target(s) in 0.03s
     Running `target/debug/examples/color --graph test.0 --nsamples 10000000 --frequency 1000000 --out colors.txt.3 --out-times times.txt.3 --seed 3`
    Finished dev [unoptimized + debuginfo] target(s) in 0.03s
     Running `target/debug/examples/color --graph test.0 --nsamples 10000000 --frequency 1000000 --out colors.txt.4 --out-times times.txt.4 --seed

In [24]:
colors = [np.loadtxt('colors.txt.{}'.format(i), dtype=int) for i in range(1, 11)]
steps = [c[:,0] for c in colors]
colors = [c[:, 1:] for c in colors]
times = [np.loadtxt('times.txt.{}'.format(i)).ravel() for i in range(1, 11)]

In [27]:
ncaptures, nvertices = colors[0].shape
for i in range(1, ncaptures):
    
    dists = []
    for c in colors:
        # in principle glauber could find a better coloring than the initial greedy one
        # but the probability of that is vanishingly small
        assert len(np.unique(c[0])) <= len(np.unique(c[i]))

        # If we could assign each color from colors[i] to one from colors[0]
        # then what vertices would still be "forced" to be different?

        # use sorting to identify the optimal assignment
        lex = c[[i, 0]]
        ixs = np.lexsort(lex)

        assignment = np.zeros(nvertices, dtype=int) - 1
        current_key = -1
        current_val = -1
        current_cnt = 0
        current_best_val = -1
        current_best_cnt = 0
        for r in ixs:
            key = lex[0,r]
            val = lex[1,r]
            if key != current_key:
                if current_key >= 0:
                    assignment[current_key] = current_best_val
                current_key = key
                current_best_val = current_val = val
                current_best_cnt = current_cnt = 1
            else:
                if current_val == val:
                    current_cnt += 1
                if current_cnt > current_best_cnt:
                    current_best_val = val
                    current_best_cnt = current_cnt
        assignment[current_key] = current_best_val

        remapped = assignment[c[i]]
        assert np.all(remapped >= 0)

        dist = (remapped != c[0]).sum() / nvertices
        dists.append(dist)
    print(steps[0][i], dist)


100000 0.51
200000 0.52
300000 0.42
400000 0.53
500000 0.5
600000 0.58
700000 0.53
800000 0.48
900000 0.57
1000000 0.56


In [2]:
import scipy
scipy.__version__

'1.6.1'